# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
#from sqlalchemy import create_engine
import sqlalchemy as sql


import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer


from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, precision_recall_fscore_support, precision_score, recall_score, f1_score

from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fsteppich\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fsteppich\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fsteppich\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = sql.create_engine('sqlite:///DisasterMessageClassification.sqlite')
with engine.connect() as conn, conn.begin():
    df = pd.read_sql_table('Messages', conn)
    #conn.commit()
    conn.close()
    
print("Data loaded ... DataFrame contains: {}".format(df.shape))

target_column_labels = set(df.columns.tolist()).difference({'id', 'message', 'original', 'genre'})
print("Target Columns ({}): {}".format(len(target_column_labels), target_column_labels))

X = df['message']
Y = df[list(target_column_labels)]

df.describe()

Data loaded ... DataFrame contains: (26176, 40)
Target Columns (36): {'earthquake', 'hospitals', 'floods', 'direct_report', 'other_infrastructure', 'shops', 'other_aid', 'fire', 'related', 'food', 'child_alone', 'storm', 'search_and_rescue', 'water', 'shelter', 'medical_products', 'electricity', 'money', 'cold', 'transport', 'other_weather', 'buildings', 'missing_people', 'security', 'request', 'death', 'infrastructure_related', 'refugees', 'military', 'aid_related', 'tools', 'weather_related', 'clothing', 'aid_centers', 'offer', 'medical_help'}


,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26176.000000,26176.000000,26176.000000,26176.000000,26176.000000,26176.000000,26176.000000,26176.000000,26176.000000,26176.000000,...,26176.000000,26176.000000,26176.000000,26176.000000,26176.000000,26176.000000,26176.000000,26176.000000,26176.000000,26176.000000
mean,15226.183985,0.766427,0.170538,0.004470,0.414120,0.079462,0.050046,0.027659,0.017994,0.032816,...,0.011805,0.043972,0.278347,0.082098,0.093215,0.010773,0.093674,0.020171,0.052567,0.193421
std,8827.169602,0.423112,0.376112,0.066708,0.492579,0.270464,0.218044,0.163997,0.132930,0.178159,...,0.108008,0.205036,0.448194,0.274519,0.290739,0.103236,0.291380,0.140588,0.223172,0.394988
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7448.750000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15663.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22925.250000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # normalize case
    tokens = word_tokenize(text.lower())
    # remove leading/trailing white space
    tokens = [t.strip() for t in tokens]
    # remove stop words
    tokens = [t for t in tokens if t not in stopwords.words("english")]
        
    # lemmatize 
    lemmatizer = WordNetLemmatizer()
    # iterate through each token
    clean_tokens = []
    for tok in tokens:    
        clean_tok = lemmatizer.lemmatize(tok, pos='v')
        clean_tokens.append(clean_tok)

    return clean_tokens


In [85]:
message = df['message'][0]
print(message)
print(tokenize(message))
print()

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', '-', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [42]:
#pipeline = Pipeline([
#    ('features', FeatureUnion([
#
#        ('text_pipeline', Pipeline([
#            ('vect', CountVectorizer(tokenizer=tokenize)),
#            ('tfidf', TfidfTransformer())
#        ])),
#
#        ('starting_verb', StartingVerbExtractor())
#    ])),
#
#    ('clf', RandomForestClassifier())
#])

from sklearn.metrics import fbeta_score, make_scorer

target_column_label_list = list(target_column_labels)

def my_custom_loss_func(y_true, y_pred):
    print("[*] my_custom_loss_func")
    # assumung 1 true and 0 false
    support_total = 0
    precision_total = 0
    for col in range(0, len(target_column_label_list)):
        col_header = target_column_label_list[col]
        test = y_true[col_header]
        pred = y_pred[:, col]
        
        support = test.sum()
        support_total += support
        
        pred_labels = np.unique(pred)
        if( len(pred_labels) != 2 ):
            print("[w] Predicted only {} in column {}. Recall and F-Score are ill-defined; setting to 0.0".format(pred_labels, col_header))
        
        precision_total += (precision_score(test, pred, labels=pred_labels, zero_division=1) * support)
        
    return precision_total / support_total

def build_pipeline(grid_search=False):
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', DecisionTreeClassifier())
            # MultiOutputClassifier(KNeighborsClassifier()) RandomForestClassifier(n_estimators=30, verbose=1)
    ])
    
    if( grid_search ):
        scorer = make_scorer(my_custom_loss_func, greater_is_better=False)
        return GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=1, cv=2, scoring=scorer)
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [112]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

# train classifier
pipeline = build_pipeline()
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x0000017C058B4DC0>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='au

In [113]:
print(len(X_train))
print(len(X_test))

19632
6544


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [114]:
# predict on test data
Y_pred = pipeline.predict(X_test)
print(len(Y_pred))

# display results
#display_results(y_test, y_pred)

print(classification_report(Y_test, Y_pred))

6544
              precision    recall  f1-score   support

           0       0.68      0.46      0.55      2678
           1       0.63      0.23      0.33       556
           2       0.66      0.11      0.19       313
           3       0.28      0.05      0.08       890
           4       0.75      0.11      0.20       212
           5       0.68      0.15      0.25       280
           6       0.42      0.02      0.04       272
           7       0.52      0.13      0.21       307
           8       0.50      0.06      0.10       122
           9       0.63      0.33      0.43       674
          10       0.67      0.17      0.27       532
          11       0.00      0.00      0.00        37
          12       0.00      0.00      0.00         0
          13       0.51      0.12      0.19       493
          14       0.32      0.04      0.08       209
          15       0.63      0.09      0.15       298
          16       0.82      0.94      0.87      4982
          17       1.0

c:\program files\python38\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\program files\python38\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\program files\python38\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\program files\python38\lib\site-packages\sklearn\metrics\

In [9]:
def print_classification_report(Y_test, Y_pred, target_column_label_list): 
    Y_pred_array = np.array(Y_pred)
    
    print("   Category               | Precision | Recall | F1 ")
    print("----------------------------------------------------")
    for col in range(0, len(target_column_label_list)):
        test = Y_test[target_column_label_list[col]]
        pred = Y_pred_array[:, col]
        
        precision = precision_score(test, pred)
        recall = recall_score(test, pred)
        f1 = f1_score(test, pred)

        
        print("{:2} {:22} | {:9.3} | {:6.3} | {:.3%}".format(col, \
                                                      target_column_label_list[col], \
                                                      precision, \
                                                      recall, \
                                                      f1
                                                     ))
        
    
#i = 16
#print(Y_pred)
#Y_pred_array = np.array(Y_pred)
#print(Y_test[target_column_label_list[i]])
#print(Y_pred_array[:, i])
#target_column_label_list = list(target_column_labels)
#print(target_column_label_list[i])
#report = classification_report(\
#                            Y_test[target_column_label_list[i]], \
#                            Y_pred_array[:, i])
#print(report)
#precision, recall, f1, support = precision_recall_fscore_support(\
#                            Y_test[target_column_label_list[i]], \
#                            Y_pred_array[:, i])

#precision = precision_score(\
#                            Y_test[target_column_label_list[i]], \
#                            Y_pred_array[:, i])
#recall = recall_score(\
#                            Y_test[target_column_label_list[i]], \
#                            Y_pred_array[:, i])
#f1 = f1_score(\
#                           Y_test[target_column_label_list[i]], \
#                            Y_pred_array[:, i])

#print("1| Precision: {}, Recall {}, F1 {}, Support {}".format(precision, recall, f1, support))
# precisionfloat (if average is not None) or array of float, shape = [n_unique_labels]
# recallfloat (if average is not None) or array of float, , shape = [n_unique_labels]
# fbeta_scorefloat (if average is not None) or array of float, shape = [n_unique_labels]
# supportNone (if average is not None) or array of int, shape = [n_unique_labels]

#precision, recall, f1, support = precision_recall_fscore_support(\
#                            Y_test[target_column_label_list[i]], \
#                            Y_pred_array[:, i])
#print("2| Precision: {}, Recall {}, F1 {}, Support {}".format(precision, recall, f1, 'xxx'))


print_classification_report(Y_test, Y_pred, target_column_label_list)

NameError: name 'target_column_label_list' is not defined

In [120]:
pred = pipeline.predict(pd.Series(["we desperatly clothe"])).tolist()
print(pred)
for i in range(0,len(pred[0])):
    if( pred[0][i] ):
        print(target_column_label_list[i])

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
aid_related
related
clothing


### 6. Improve your model
Use grid search to find better parameters. 

In [6]:
pipeline = build_pipeline()
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x0000029A8B66B790>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                          criterion='gini', max_depth=None, max_features='auto',
                          max_leaf_nodes=None, max_samples=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,

In [43]:
parameters = {
#        'vect__ngram_range': ((1, 1), (1, 2)),
#        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_df': (0.5,.75)
#        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
#        'features__text_pipeline__tfidf__use_idf': (True, False),
#        'clf__n_estimators': [50, 100, 200]
#        'clf__min_samples_split': [2, 3, 4],
#        'features__transformer_weights': (
#            {'text_pipeline': 1, 'starting_verb': 0.5},
#            {'text_pipeline': 0.5, 'starting_verb': 1},
#            {'text_pipeline': 0.8, 'starting_verb': 1},
#        )
    }

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

# Increased number of jobs (default = None)
# Verbosity increased (value greater -> more output)
pipeline = build_pipeline(grid_search=True)

# train classifier
pipeline.fit(X_train, Y_train)
Y_pred = pipeline.predict(X_test)

print ("Grid best score: ", pipeline.best_score_)


print(classification_report(Y_test, Y_pred))
print_classification_report(Y_test, Y_pred, list(target_column_labels))

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] vect__max_df=0.5 ................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[*] my_custom_loss_func
[w] Predicted only [0] in column child_alone. Recall and F-Score are ill-defined; setting to 0.0
[CV] ................................. vect__max_df=0.5, total= 7.0min
[CV] vect__max_df=0.5 ................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.0min remaining:    0.0s


[*] my_custom_loss_func
[w] Predicted only [0] in column child_alone. Recall and F-Score are ill-defined; setting to 0.0
[CV] ................................. vect__max_df=0.5, total= 7.0min
[CV] vect__max_df=0.75 ...............................................
[*] my_custom_loss_func
[w] Predicted only [0] in column child_alone. Recall and F-Score are ill-defined; setting to 0.0
[CV] ................................ vect__max_df=0.75, total= 6.7min
[CV] vect__max_df=0.75 ...............................................
[*] my_custom_loss_func
[w] Predicted only [0] in column child_alone. Recall and F-Score are ill-defined; setting to 0.0
[CV] ................................ vect__max_df=0.75, total= 6.7min


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 27.5min finished


Grid best score:  -0.5786957122799979
              precision    recall  f1-score   support

           0       0.03      0.02      0.02       124
           1       0.12      0.06      0.08        65
           2       0.25      0.17      0.20       133
           3       0.62      0.60      0.61       409
           4       0.26      0.15      0.19       227
           5       0.27      0.19      0.22       495
           6       0.31      0.26      0.28       845
           7       0.13      0.07      0.09       285
           8       0.63      0.49      0.55       636
           9       0.16      0.12      0.14       132
          10       0.44      0.31      0.36       110
          11       0.14      0.09      0.11       291
          12       0.26      0.13      0.17        76
          13       0.76      0.74      0.75       611
          14       0.18      0.12      0.14       170
          15       0.61      0.48      0.54       528
          16       0.14      0.08      0.10

c:\program files\python38\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\program files\python38\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\program files\python38\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\program files\python38\lib\site-packages\sklearn\metrics\

12 fire                   |     0.256 |  0.132 | 17.391%
13 earthquake             |     0.755 |  0.738 | 74.669%
14 search_and_rescue      |     0.182 |  0.118 | 14.286%
15 floods                 |     0.611 |  0.479 | 53.715%
16 infrastructure_related |      0.14 |  0.076 | 9.861%
17 other_weather          |     0.174 |  0.119 | 14.094%
18 direct_report          |     0.476 |  0.437 | 45.566%
19 buildings              |     0.364 |  0.214 | 26.945%
20 related                |     0.862 |  0.836 | 84.847%
21 request                |     0.557 |  0.537 | 54.702%
22 hospitals              |    0.0488 | 0.0357 | 4.124%
23 shops                  |    0.0714 |  0.027 | 3.922%
24 child_alone            |       0.0 |    0.0 | 0.000%
25 tools                  |       0.0 |    0.0 | 0.000%
26 weather_related        |     0.719 |  0.622 | 66.686%
27 medical_products       |     0.272 |  0.199 | 23.006%
28 shelter                |     0.525 |  0.451 | 48.520%
29 aid_centers            |    0.042

c:\program files\python38\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\program files\python38\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\program files\python38\lib\site-packages\sklearn\metrics\_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


In [44]:
print(classification_report(Y_test, Y_pred))
print_classification_report(Y_test, Y_pred, list(target_column_labels))

              precision    recall  f1-score   support

           0       0.03      0.02      0.02       124
           1       0.12      0.06      0.08        65
           2       0.25      0.17      0.20       133
           3       0.62      0.60      0.61       409
           4       0.26      0.15      0.19       227
           5       0.27      0.19      0.22       495
           6       0.31      0.26      0.28       845
           7       0.13      0.07      0.09       285
           8       0.63      0.49      0.55       636
           9       0.16      0.12      0.14       132
          10       0.44      0.31      0.36       110
          11       0.14      0.09      0.11       291
          12       0.26      0.13      0.17        76
          13       0.76      0.74      0.75       611
          14       0.18      0.12      0.14       170
          15       0.61      0.48      0.54       528
          16       0.14      0.08      0.10       421
          17       0.17    

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:

def build_pipeline(grid_search=False):
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', Bo())
        ])),
        ('clf', DecisionTreeClassifier())
            # MultiOutputClassifier(KNeighborsClassifier()) 
            # RandomForestClassifier(n_estimators=30, verbose=1)
            # DecisionTreeClassifier
    ])
    
    if( grid_search ):
        scorer = make_scorer(my_custom_loss_func, greater_is_better=False)
        return GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=1, cv=2, scoring=scorer)
    
    return pipeline
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', DecisionTreeClassifier())
    ])

### 9. Export your model as a pickle file

In [26]:
import pickle
with open('desaster-response-pipeline__vect__max_df__wavg.56.pkl', 'wb') as file:
    pickle.dump(pipeline, file)
    file.close()   

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.